In [1]:
from FedCVL import FedCVTClient, FedCVTServer

In [2]:
# ====================== HVAC CLIENT ======================
import socket, pickle
import torch
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


HOST, PORT = "localhost", 9000

# Load HVAC data
hvac_data = pd.read_csv("../data/client1_hvac.csv").values

# Split train/test (same random_state as server)
indices = np.arange(len(hvac_data))
train_idx, test_idx = train_test_split(indices, test_size=0.2, random_state=42)
hvac_train, hvac_test = hvac_data[train_idx], hvac_data[test_idx]

# Init client
hvac_client = FedCVTClient("HVAC", input_dim=hvac_data.shape[1], encoding_dim=16)

# Preprocess
hvac_train_tensor = hvac_client.preprocess_data(hvac_train, fit_scaler=True)
hvac_test_tensor = hvac_client.preprocess_data(hvac_test)

# Connect
sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
sock.connect((HOST, PORT))

num_rounds = 5
batch_size = 32
dataset_size = len(hvac_train_tensor)
num_batches = (dataset_size + batch_size - 1) // batch_size

for round_num in range(num_rounds):
    print(f"\n[HVAC] Round {round_num+1}/{num_rounds}")
    for batch_idx in range(num_batches):
        start, end = batch_idx*batch_size, min((batch_idx+1)*batch_size, dataset_size)
        batch = hvac_train_tensor[start:end]

        # Send masked encodings
        z_masked = hvac_client.send_encoded_data(batch)
        sock.send(pickle.dumps(z_masked.detach().numpy()))
        print(f"[HVAC] Sent batch {batch_idx+1}/{num_batches}")

        # Receive gradients
        grads = pickle.loads(sock.recv(65536))
        grads_tensor = torch.tensor(grads, dtype=torch.float32)

        # Update encoder
        hvac_client.local_training_step(batch)
        hvac_client.update_encoder(grads_tensor, batch)

print("\n[HVAC] Training finished.")

# --- Wait for evaluation request ---
msg = pickle.loads(sock.recv(65536))
if msg == "SEND_TEST":
    print("[HVAC] Sending test encodings to server...")
    z_test = hvac_client.send_encoded_data(hvac_test_tensor)
    sock.send(pickle.dumps(z_test.detach().numpy()))



[HVAC] Round 1/5
[HVAC] Sent batch 1/25


/media/gaurav/ubuntudata/SemThree/DML/dml_grp49/src/FedCVL.py:130: UserWarning: torch.qr is deprecated in favor of torch.linalg.qr and will be removed in a future PyTorch release.
The boolean parameter 'some' has been replaced with a string parameter 'mode'.
Q, R = torch.qr(A, some)
should be replaced with
Q, R = torch.linalg.qr(A, 'reduced' if some else 'complete') (Triggered internally at /pytorch/aten/src/ATen/native/BatchLinearAlgebra.cpp:2487.)
  rotation_matrix = torch.qr(rotation_matrix)[0]  # Orthogonal matrix


[HVAC] Sent batch 2/25
[HVAC] Sent batch 3/25
[HVAC] Sent batch 4/25
[HVAC] Sent batch 5/25
[HVAC] Sent batch 6/25
[HVAC] Sent batch 7/25
[HVAC] Sent batch 8/25
[HVAC] Sent batch 9/25
[HVAC] Sent batch 10/25
[HVAC] Sent batch 11/25
[HVAC] Sent batch 12/25
[HVAC] Sent batch 13/25
[HVAC] Sent batch 14/25
[HVAC] Sent batch 15/25
[HVAC] Sent batch 16/25
[HVAC] Sent batch 17/25
[HVAC] Sent batch 18/25
[HVAC] Sent batch 19/25
[HVAC] Sent batch 20/25
[HVAC] Sent batch 21/25
[HVAC] Sent batch 22/25
[HVAC] Sent batch 23/25
[HVAC] Sent batch 24/25
[HVAC] Sent batch 25/25

[HVAC] Round 2/5
[HVAC] Sent batch 1/25
[HVAC] Sent batch 2/25
[HVAC] Sent batch 3/25
[HVAC] Sent batch 4/25
[HVAC] Sent batch 5/25
[HVAC] Sent batch 6/25
[HVAC] Sent batch 7/25
[HVAC] Sent batch 8/25
[HVAC] Sent batch 9/25
[HVAC] Sent batch 10/25
[HVAC] Sent batch 11/25
[HVAC] Sent batch 12/25
[HVAC] Sent batch 13/25
[HVAC] Sent batch 14/25
[HVAC] Sent batch 15/25
[HVAC] Sent batch 16/25
[HVAC] Sent batch 17/25
[HVAC] Sent bat